In [39]:
from google.cloud import datastore
import datetime
import pandas

# DataStore

In [40]:
GOOGLE_APPLICATION_CREDENTIALS="/Users/Shared/digital-africa/web-services-d4-digital-africa-2eb794c55b04.json"
client = datastore.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
entity_name = 'SU_challenge1000_'
not_index = ['prez_struc', 'prez_produit_struc']

In [41]:
def create_data(client, entity_name, data, not_index):
    key = client.key(entity_name,data['key_main'])
    record = datastore.Entity(key, exclude_from_indexes=not_index)
    #task = datastore.Entity(key)
    #data.update({'created': datetime.datetime.utcnow()})
    record.update(data)
    client.put(record)
    return record.key

def update_record(client,entity_name, key_main):
    with client.transaction():
        key = client.key(entity_name, key_main)
        su = client.get(key)

        if not task:
            raise ValueError(
                'Task {} does not exist.'.format(key_main))

       #su['Agriculture'] = True

        client.put(task)

def list_records(client, entity_name):
    query = client.query(kind=entity_name)
    #query.order = ['created']
    return list(query.fetch())

def datastore_to_pandas(SU):
    #ds = list_records(client, entity_name)
    ds = [dict(a.items()) for a in SU]
    records = pandas.DataFrame.from_dict(ds_).set_index('key_main')
    return records

# Data with label

In [42]:
import chalenge1000


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out


Chalenge = chalenge1000.Native()
descriptions = Chalenge.descriptions
data = descriptions[['nom_struc','prez_struc', 'prez_produit_struc']].reset_index().to_dict(orient = 'records')
data = chunkIt(data, 10)
data_test = data[0]

In [43]:
import chalenge1000
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def to_boolean(element):
    if element == '1':
        return True
    else:
        return False
    
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'credentials.json', scope) # Your json file here

gc = gspread.authorize(credentials)

wks = gc.open("training_set").sheet1

data = wks.get_all_values()
headers = data.pop(0)

label = pandas.DataFrame(data, columns=headers).set_index('key_main')
label['producteur'] = label['producteur'].map(to_boolean)
label['plateforme'] = label['plateforme'].map(to_boolean)


len(label)

149

In [44]:
label[['producteur', 'plateforme']]

,producteur,plateforme
key_main,,
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,False,True
https://m.facebook.com/story.php?story_fbid=523711488470216&id=333247734183260_agbogan_codjomarc,False,True
https://agribusinessdata.com_depo_adebayoajadi,False,True
http://www.miensahgroup.com/_kouakou_koffigislain,False,False
http://www.mokine.org_ziegoubefoka_emmanuel,False,False
...,...,...
http://www.cbdibaong.org_houedanou_martin,True,False
http://www.cbdibaong.org_yemadje_emeline,True,False
http://www.cbdibaong.org_akokponsode_hippolyte,True,False


In [46]:
import chalenge1000
import collections
import pandas

def lw(anystring):
    try:
        return anystring.lower()
    except:
        return ''
    
def is_agriculture(element):
    if element == 'agriculture':
        return True
    else:
        return False

def _agriculture__(element):
    if element in agriculture:
        return True
    else:
        return False

df = chalenge1000.Native().ALL   
keys = df.keys().values.tolist()

cat = [a for a in keys if 'cat' in a]
for element in cat:
    df[element] = df[element].map(lw)
    
agr = {e for e in df['cat_autre_struc'].values.tolist() if 'agr' in e}
alim = {e for e in df['cat_autre_struc'].values.tolist() if 'alim' in e}
agriculture = alim.union(agr)    
df['label_agriculture'] = df['categorie'].map(is_agriculture)
df['maybe_agriculture'] = df['cat_autre_struc'].map(_agriculture__)

df = df[['label_agriculture','maybe_agriculture']].join(label[['producteur', 'plateforme']])
df = df.join(chalenge1000.Native().descriptions[['nom_struc','prez_struc', 'prez_produit_struc']])
set(df['producteur'])

{nan, False, True}

In [48]:
print(len(df[df['producteur']==True]))
print(len(df[df['plateforme']==True]))
print(len(df[df['label_agriculture']==True]))
print(len(df[df['maybe_agriculture']==True]))

19
58
697
15


# Write to datastore

In [49]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

data = df.reset_index().to_dict(orient = 'records')
data = chunkIt(data, 10)

In [50]:
def batch_write():
    #e = client.query(kind='SU_challenge1000').fetch()
    #[client.delete(k.key) for k in list(e)]
    for chunk in data:
        batch = client.batch()
        with batch:
            for element in chunk:
                create_data(client, entity_name, element, not_index)
batch_write()

In [118]:
len(list_records(client, entity_name))

3449

# Read from datastore

In [51]:
ds = list_records(client, entity_name)
datastore_to_pandas(ds)

,_agriculture_,prez_struc,producteur,nom_struc,label_agriculture,plateforme,prez_produit_struc
key_main,,,,,,,
http://....._makinita_johannesmatome,False,Reetsang-Kedi Holdings is a consulting firm sp...,NaN,Reetsang-Kedi Consulting Agency (Pty) Ltd,True,NaN,"Biosecurity - This includes pest control, hou..."
http://0sachetplastique.unblog.fr_guebre_daouda,False,Une structure de recyclage de déchets plastiques.,NaN,0 déchet plastique,False,NaN,"En recyclant, les déchets plastiques en poudre."
http://17goalsafrica.org_oluokun_kayodeabiodun,False,17GoalsAfrica Initiative is an organisation se...,NaN,17 Goals Africa Initiative,False,NaN,United Nations Women Global Goals World Cup. I...
http://2htechnologie.com_camara_hassenmohamed,False,2H-Technologie est une entreprise spécialisée ...,NaN,2H-TECHNOLOGIE,False,NaN,vente du Materiel Informatique et VIDÉO-SURVEI...
http://7WWW.info.ci/_kabore_jeanphilippe,False,Optimum Médias est la structure qui porte la m...,NaN,Optimum Media,False,NaN,Le projet phare est d’apporter une information...
...,...,...,...,...,...,...,...
https://yolibrand.com/_souaga_etransilvanus,False,Société à Responsabilité Limitée au capital de...,NaN,YOLIBRAND,False,NaN,Site de personnalisation en ligne. Permettant ...
https://yonja.website/_andrew_tugume,False,Yonja is a sanitation service delivery company...,NaN,Yonja Mobile Solutions Ltd,False,NaN,A GIS enabled and data driven mobile applicat...
https://youtu.be/1C2ZpqDTO34_lumono_perry,False,Nous sommes un groupe de chercheurs de plus d'...,NaN,GROUPE AGENCE MODERNE,False,NaN,notre entreprise résout le problème des déchet...


In [33]:
query = client.query(kind=entity_name)
query.add_filter('_agriculture_', '=', True)
[dict(e.items()) for e in query.fetch()]

[{'key_main': 'http://YamAgroIndustrie.bf_yameogo_stevearmandleonardwendkouni',
  'label_agriculture': False,
  'nom_struc': 'Yam agro INDUSTRIE',
  'plateforme': nan,
  'prez_produit_struc': "Nous avons deux types de sirop à savoir le sirop de bissap et de gingembre. Un biscuit les cookies au chocolat et une confiture celle à la mangue. Nous envisageons mettre sur le marché d'ici mars 2020 trois nouveaux sirops et une confiture",
  'prez_struc': 'Entreprise spécialisée dans la transformation agroalimentaire made in Burkina Faso,  nous proposons aux consommateurs burkinabé des sirop , des biscuits et de la confiture.',
  '_agriculture_': True,
  'producteur': nan},
 {'producteur': nan,
  'key_main': 'http://www.aquareignindustry.com_mbouleediango_marcidriss',
  'label_agriculture': False,
  'nom_struc': 'Aquareign industry',
  'plateforme': nan,
  'prez_produit_struc': 'Etant donné que notre projet porte toute une chaine de valeur, alors nous disposons de plusieurs produits et services

In [54]:
datastore_to_pandas(ds).to_csv('datastore.csv')

In [56]:
for e in agriculture:
    print(e)

industrie agroalimentaire
agro industrie
alimentations, distribution et services
agroalimentaire - nutrition
transformation  agro-alimentaire
agroalimentaire (pâtisseries sans gluten à base des tubercules et céréales)
agriculture en zone urbaine
ecologie environnement changement climatique conjoint à l'agriculture hors sol
artisanat, cosmétique, agro-transformation et nouvelle technologie(digital)
aquaculture, pisciculture urbaine, sauvegarde de la biodiversité et sécurité alimentaire.
financement de la production et la transformation agricole, financement des jeunes urbain pour des initiatives économiques
agroalimentaire
industrie agro alimentaire
transformation alimentaire  ( produits à base d'insectes)
transformation agroalimentaire
